# Exercise 8: Dimensionality Reduction
In this exercise, we will implement and see the working of a dimensionality reduction techniques Prinical Component Analysis (PCA) and Linear Discriminant Analysis (LDA) . 

In [ ]:
# good to import few packages
%matplotlib notebook
from ipywidgets import *
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from mpl_toolkits import mplot3d
from sklearn import datasets
from sklearn.datasets import fetch_olivetti_faces

## 1. Toy Dataset
Let see the PCA results on a toy iris dataset.

In [ ]:
# load iris dataset
iris = datasets.load_iris()
data = iris['data'].astype(np.float32) 
labels = iris['target'] 
cls_names = iris['target_names']

Visualize the trends of different features together. One can see that one class is well separated from others.

In [ ]:
plt.figure(figsize=(12,8))
count = 1
colors = np.array([[0.85, 0.85, 0], [0, 0.5, 0], [0.25, 0.25, 1]])
for i in range(3):
    for j in range(i+1,4):
        plt.subplot(2,3,count)
        for ind,name in enumerate(cls_names):
            filtered_class = labels==ind
            plt.scatter(data[filtered_class,i],data[filtered_class,j],c=colors[ind,None],label=name)
        plt.xlabel(f'feature_{i}')
        plt.ylabel(f'feature_{j}')
        plt.legend()
        count +=1



## 2. PCA
In the above dataset, we have 4 features per data point and now let's try to reduce it dimensionality to 2 using PCA. Implement the following PCA function.


For a dataset $\mathbf{X}\in R^{N\times D}$, PCA solves follwing optimization problem
   \begin{align}
    \underset{\mathbf{W}}{\operatorname{max}} \mathbf{W}^T\mathbf{C}\mathbf{W}\\
    s.t. ~~~~~ \mathbf{W}^T\mathbf{W} = \mathbf{I_d}
   \end{align}
   
 where data convariance matrix
     \begin{align}
         \mathbf{C} &= \frac{1}{N}\sum_{i=0}^{N-1}(\mathbf{x_i}-\mathbf{\bar{x}})(\mathbf{x_i}-\mathbf{\bar{x}})^T\\
         \mathbf{\bar{x}} &= \frac{1}{N}\sum_{i=0}^{N-1} \mathbf{x_i}
     \end{align}
     
 and $\mathbf{W}\in R^{D\times d}$, $\mathbf{x}\in R^{D\times 1}$, $\mathbf{\bar{x}}\in R^{D\times 1}$
 
 The solution to this problem is finding eigenvectors for $d(<D)$ largest eigenvalues of data covariance matrix $\mathbf{C}$. Hence, $\mathbf{W}$ is a matrix of $d$ eigenvectors each being $D$-dimensional. 
 
We project our original data $\mathbf{X}\in R^{N\times D}$ space to $R^{N\times d}$, using centered data $\mathbf{\tilde{X}}\in R^{N\times D}$,
    \begin{align}
        \mathbf{\hat{X}} &= \mathbf{\tilde{X}}\mathbf{W} \\
        \mathbf{\tilde{X}(i)} &= \mathbf{x_i}-\mathbf{\bar{x}} ~~~~ i\in \[ 0,N-1\]
    \end{align}
 
To understand how much of variance is explained by our $d$ eigenvectors, we compute percentage of variance explained by 
    \begin{align}
        \mathbf{exvar} = \frac{\sum_{i=0}^{d-1}\lambda_i}{\sum_{i=0}^{D-1}\lambda_i}
    \end{align}
where $\lambda_i$ is the ith largest eigenvalue. For different applications, you would like to choose $d$ such that explained variance is greater than a threshold.

In [ ]:
'''
Input:
    X: NxD matrix representing our data
    d: Number of principal components to be used to reduce dimensionality
    
Output:
    mean_data: 1xD representing the mean of input data
    W: Dxd principal components
    eg: d values representing variance corresponding to principal components
    X_hat: Nxd data projected in principal components' direction
    exvar: explained variance by principal components
'''
def PCA(X, d):
    
    # Compute the mean of data
    mean = 
    # Center the data with the mean
    X_tilde = 
    # Create covariance matrix
    C = 
    # Compute eigenvector and eigenvalues. Hint use: np.linalg.eigh
    eigvals, eigvecs = 
    # Choose top d eigenvalues and corresponding eigenvectors. Sort eigenvalues( with corresponding eigenvector )
    # in decreasing order first.
    eigvals =
    eigvecs = 

    W = eigvecs[:, 0:d]
    eg = eigvals[0:d]

    # project the data using W
    X_hat = 
    
    #explained variance
    exvar = 

    return mean, W, eg, X_hat, exvar

Let's call the implemented function and visualize the projected data

In [ ]:
d = 2
mean, W, eg, X_hat, exvar = PCA(data, d)
print(f'Total Variance explained by first {d} pc is {exvar}')

In [ ]:
plt.figure()
for ind,name in enumerate(cls_names):
    filtered_class = labels==ind
    plt.scatter(X_hat[filtered_class,0],X_hat[filtered_class,1],c=colors[ind,None],label=name)
plt.xlabel(f'feature_0')
plt.ylabel(f'feature_1')
plt.legend()

**Q.** What happens when d=D?  

**Q.** What happens when D>>N?  


## 3. EigenFaces
Now, we will use PCA on face of images. The goal is to represent faces in the dataset with set of faces, called eigenfaces. 

In [ ]:
faces = fetch_olivetti_faces().data
print(f'Dimensions of Face dataset N={faces.shape[0]}, D={faces.shape[1]}')

Try different values of d and see the variance explained

In [ ]:
d = 30
mean, W, eg, X_hat, exvar = PCA(faces, d)
print(f'Total Variance explained by first {d} pc is {exvar}')

### 3.1 Visualize
Let see what do these principal component look like.

In [ ]:
plt.figure()
plt.imshow(mean.reshape(64,64),cmap='gray')
plt.title('Mean Face')

In [ ]:
# see first 10 principal components
plt.figure(figsize=(8,18))
for i in range(10):
    plt.subplot(5,2,i+1)
    plt.imshow(W.reshape(64,64,-1)[...,i],cmap='gray')
    plt.xlabel(f'Principal Component:{i}')

Observe what these components account for. Vary the slider to change the principal component and their influence on the mean value. 

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
line = ax.imshow(mean.reshape(64,64),cmap='gray')

def update(pcind = 0,pcweight=0):
    img = W.copy()[:,pcind]*pcweight
    line.set_data((img+mean).reshape(64,64))
    fig.canvas.draw_idle()

interact(update,pcind=(0,d-1,1),pcweight=(-10,10,1));

**Q.** Can you identify what component accounts for what?  


### 3.2 Reconstruction
We project our original data to smaller dimension and depending upon how many dimension are kept, we will have some loss of information. Here we will see how changing final number of dimensions our reconstruction is affected. Also relate this to the total variance explained by the principal components.

In [ ]:
# Try different values of d
d = 10
mean, W, eg, X_hat, exvar = PCA(faces, d)
print(f'Total Variance explained by first {d} pc is {exvar}')

In [ ]:
sample_id = np.random.choice(faces.shape[0],1)[0]
sample_face = faces[sample_id,:]
#project this face in smaller dimension. remember to center the data first
proj_face = 
#bring to back to original space
reconstructed_face = 

In [ ]:
plt.figure()
ax = plt.subplot(1,2,1)
plt.imshow(sample_face.reshape(64,64),cmap='gray')
ax.set_title('Original Image')
ax = plt.subplot(1,2,2)
plt.imshow(reconstructed_face.reshape(64,64),cmap='gray')
ax.set_title('Reconstructed Image')

## 4. Fisher Linear Discriminant Analysis
This supervised method is used to reduce dimensionality along with learning a projection, which keeps data points belonging to same class together. We will use sklearn's implementation of the Fisher LDA to project MNIST data to smaller dimensions.

In [ ]:
#load MNIST data
mnist = datasets.load_digits()
data = mnist.data
labels = mnist.target
num_class = 10

Project to 2 dimensional space

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
# see the documentation of the method to understand the parameters
d = 2
clf = LDA(n_components=d)
# call the fit function
obj = 
# computed the variance explained using clf's parameter
exvar = 
print(f'Variance explained {exvar}')
proj = obj.transform(data)

In [ ]:
plt.figure()
colors = cm.jet(np.linspace(0, 1, num_class))
for i in range(num_class):
    inds = labels == i
    plt.scatter(proj[inds,0],proj[inds,1],c=colors[i])
plt.legend(np.arange(num_class))

Projection in 3 dimensional space

In [ ]:
d = 3
clf = LDA(n_components=d)
# call the fit function
dd = 
proj = dd.transform(data)
# computed the variance explained using clf's parameter
exvar = 
print(f'Variance explained {exvar}')

In [ ]:
plt.figure()
ax = plt.axes(projection='3d')
for i in range(num_class):
    inds = labels == i
    ax.scatter3D(proj[inds,0], proj[inds,1], proj[inds,2],c=colors[i])
plt.legend(np.arange(num_class))
plt.title('Drag the plot see the clusters')